# Iterative Method for Solving Linear Algebraic Equations

Before we consider specific methods to solve systems of linear algebraic equations, such as:

* __Jacobi Method__: The Jacobi method is an iterative solver that, starting from an initial guess, repeatedly refines each unknown in parallel using the residual between the right-hand side and the contributions of the other variables. It’s easy to implement and parallelize, and converges when the system matrix satisfies appropriate conditions (e.g., strict diagonal dominance).
* __Gauss–Seidel Method__: The Gauss–Seidel method is an iterative solver that, starting from an initial guess, refines each unknown one at a time using the most recent updates, feeding back new values immediately. This change typically yields faster convergence than Jacobi under the same matrix conditions. It’s simple to implement but inherently sequential, and converges when the system matrix is, for example, strictly diagonally dominant.
* __Successive Over-Relaxation (SOR) Method__: The SOR method builds on Gauss–Seidel by introducing a relaxation factor $\omega\in(0,2)$ that _over-relaxes_ each update, i.e., it blends the new Gauss–Seidel value with the old iterate to accelerate convergence further. With a well-chosen $\omega$, SOR can dramatically speed up solving diagonally-dominant systems, with convergence guaranteed for $0<\omega<2$.

Let's step back and sketch out a general method, then we'll discuss each specific method in different lessons.

## General
Suppose we have a _square system_ of linear equations represented in matrix form as $\mathbf{A}\mathbf{x} = \mathbf{b}$,
where the system matrix $\mathbf{A}\in\mathbb{R}^{n \times n}$, the unknown vector is $\mathbf{x}\in\mathbb{R}^{n}$, and $\mathbf{b}\in\mathbb{R}^{n}$ is the right-hand side vector. Our goal is to _iteratively_ find a vector $\mathbf{x}$ that _satisfies (in some sense)_ this equation.

Let's sketch the steps of a general iterative method for solving this linear system.

__Initialize__: Given the system matrix $\mathbf{A}\in\mathbb{R}^{n \times n}$ and the right-hand side vector $\mathbf{b}\in\mathbb{R}^{n}$, we start with an initial guess for the solution vector $\mathbf{x}^{(0)}\in\mathbb{R}^{n}$. This guess can be a zero vector or any other reasonable approximation. Set $\texttt{converged} \gets \texttt{false}$ and the iteration counter $k\gets0$. Specify a convergence criterion, such as a tolerance level $\epsilon > 0$, and a maximum number of iterations $\texttt{max\_iter}$.

While not $\texttt{converged}$ __do__:
1. Calculate the residual vector $\mathbf{r}^{(k)} \gets \mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}$.
2. Check for convergence:
   - If $\|\mathbf{r}^{(k)}\|_{2} < \epsilon$, __then__: set $\texttt{converged} \gets \texttt{true}$. 
   - If $k \geq \texttt{max\_iter}$, __then__: set $\texttt{converged} \gets \texttt{true}$, and print a __warning__ that the method did not converge.
3. Calculate an update direction. Split $\mathbf{A} = \mathbf{M} - \mathbf{N}$, where $\mathbf{M}$ is a matrix that can be easily inverted (e.g., diagonal or triangular), and $\mathbf{N}$ is the remaining part of the matrix. The update direction can then be computed as: $\mathbf{d}^{(k)} \gets \mathbf{M}^{-1}\mathbf{r}^{(k)}$.
4. Update the solution vector: $\mathbf{x}^{(k+1)} \gets \mathbf{x}^{(k)} + \mathbf{d}^{(k)}$.
5. Increment the iteration counter: $k \gets k + 1$.

___

## Update direction
The magic of iterative methods lies in the choice of the update direction. But where does this update direction come from? In the general case, we can think of the update direction as being derived from the residual vector $\mathbf{r}^{(k)}$ and the system matrix $\mathbf{A}$. 

Starting from the original system:
$$
\begin{align*}
\mathbf{A}\;\mathbf{x} &= \mathbf{b}\quad\mid\text{substitute}\;\mathbf{A} = \mathbf{M} - \mathbf{N} \\
(\mathbf{M} - \mathbf{N})\;\mathbf{x} &= \mathbf{b} \\
\mathbf{M}\mathbf{x}^{(k+1)} - \mathbf{N}\mathbf{x}^{(k)} &= \mathbf{b}\quad\mid\text{solve for}\;\mathbf{x}^{(k+1)}\\
\mathbf{x}^{(k+1)} &= \mathbf{M}^{-1}(\mathbf{b} + \mathbf{N}\mathbf{x}^{(k)})\quad\mid\text{substitute}\;\mathbf{M}^{-1}\mathbf{N} = \mathbf{I} - \mathbf{M}^{-1}\mathbf{A}\\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{M}^{-1}\underbrace{(\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)})}_{\text{residual}\;\mathbf{r}^{(k)}}\\
\mathbf{x}^{(k+1)} & = \mathbf{x}^{(k)} + \underbrace{\mathbf{M}^{-1}\mathbf{r}^{(k)}}_{\text{direction}\;\mathbf{d}^{(k)}}\\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{d}^{(k)}\quad\blacksquare\\
\end{align*}
$$

The update direction $\mathbf{d}^{(k)}$ is derived from the residual vector $\mathbf{r}^{(k)}$ and the system matrix $\mathbf{A}$. The choice of $\mathbf{M}$ and $\mathbf{N}$ determines the specific iterative method being used, such as Jacobi, Gauss-Seidel, or Successive Over-Relaxation (SOR).

___

## Convergence
One common question that arises when using iterative methods is: _When does the method converge?_ In other words, how do we know that the sequence of iterates $\{\mathbf{x}^{(k)}\}$ will approach the true solution $\mathbf{x}^{\star}$ as $k$ increases? A stationary iteration $\mathbf{x}^{(k+1)}=\mathbf{G}\,\mathbf{x}^{(k)}+\mathbf{c}$ converges __for every__ $\mathbf{x}^{(0)}$ __if and only if__:
$$
\rho(\mathbf{G}) = \rho\bigl(\mathbf{M}^{-1}\mathbf{N}\bigr) \;<\;1.
$$
The spectral radius $\rho(\mathbf{G}) = \;\max_i|\lambda_i|$, where $\lambda_i$ are the eigenvalues of the iteration matrix $\mathbf{G}$. This means that the method will converge to the true solution regardless of the initial guess $\mathbf{x}^{(0)}$.

### Where does this come from?
The convergence of iterative methods depends on the properties of the system matrix $\mathbf{A}$, and particularly on the matrix product $\mathbf{G} = \mathbf{M}^{-1}\mathbf{N}$ (iteration matrix). Let's dig into the details:
$$
\begin{align*}
\mathbf{x}^{(k+1)} & = \mathbf{M}^{-1}\;(\mathbf{b} + \mathbf{N}\;\mathbf{x}^{(k)})\\
\mathbf{x}^{(k+1)} & = \mathbf{M}^{-1}\;\mathbf{b} + \underbrace{\mathbf{M}^{-1}\mathbf{N}}_{\mathbf{G}}\;\mathbf{x}^{(k)}\\
\mathbf{x}^{(k+1)} & = \underbrace{\mathbf{M}^{-1}\;\mathbf{b}}_{\mathbf{c}} + \mathbf{G}\;\mathbf{x}^{(k)}\\
\mathbf{x}^{(k+1)} & = \mathbf{c} + \mathbf{G}\;\mathbf{x}^{(k)}\quad\blacksquare\\
\end{align*}
$$
At the _true_ solution $\mathbf{x}^{\star}$, we know that:
$$
\begin{align*}
\mathbf{x}^{\star} & = \mathbf{c} + \mathbf{G}\;\mathbf{x}^{\star}\\
\mathbf{x}^{\star} - \mathbf{G}\;\mathbf{x}^{\star} & = \mathbf{c}\\
(\mathbf{I} - \mathbf{G})\;\mathbf{x}^{\star} & = \mathbf{c}\\
\mathbf{x}^{\star} & = (\mathbf{I} - \mathbf{G})^{-1}\;\mathbf{c}\quad\blacksquare\\
\end{align*}
$$

Let's define the error vector as $\mathbf{e}^{(k+1)} = \mathbf{x}^{(k+1)} - \mathbf{x}^{\star}$. Then we can express the error in terms of the iteration matrix $\mathbf{G}$:
$$
\begin{align*}
\mathbf{e}^{(k+1)} & = \mathbf{x}^{(k+1)} - \mathbf{x}^{\star}\\
& = \left(\mathbf{c} + \mathbf{G}\;\mathbf{x}^{(k)}\right) - \left(\mathbf{c} + \mathbf{G}\;\mathbf{x}^{\star}\right)\\
& = \mathbf{G}\;\mathbf{x}^{(k)} - \mathbf{G}\;\mathbf{x}^{\star}\\
& = \mathbf{G}\;\left(\mathbf{x}^{(k)} - \mathbf{x}^{\star}\right)\\
& = \mathbf{G}\;\mathbf{e}^{(k)}\quad\blacksquare\\
\end{align*}
$$
Then we know that the error as we march through the iterations is given by:
$$
\begin{align*}
\mathbf{e}^{(1)} & = \mathbf{G}\;\mathbf{e}^{(0)}\\
\mathbf{e}^{(2)} & = \mathbf{G}^{2}\;\mathbf{e}^{(0)}\\
\vdots & \\
\mathbf{e}^{(k)} & = \mathbf{G}^{k}\;\mathbf{e}^{(0)}\quad\blacksquare\\
\end{align*}
$$
where $\mathbf{e}^{(0)}$ is the initial error vector, and $\mathbf{G}^k$ is the iteration matrix raised to the power of $k$. Ultimately, we want $\mathbf{e}^{(k)}$ to converge to zero as $k$ increases. Thus, this requires $\mathbf{G}^{k}\rightarrow 0$ as $k\rightarrow \infty$; this only occurs when $\rho(\mathbf{G}) < 1$.

### Diagonal dominance?
When we introduced this module, we mentioned the idea of diagonal dominance, but we developed our convergence condition in terms of the spectral radius. Are these two things related in some way? 

Recall that we split the matrix $\mathbf{A}$ into its diagonal and off-diagonal components: $\mathbf{A} = \mathbf{M} - \mathbf{N}$, where $\mathbf{M}$ is the diagonal part of $\mathbf{A}$ and $\mathbf{N}$ is the off-diagonal part. The iteration matrix is then given by: $\mathbf{G} = \mathbf{M}^{-1}\mathbf{N}$. Thus, for a matrix $\mathbf{A}$ to be strictly diagonally dominant, we require that for each row $i$:
$$
    |\mathbf{M}_{ii}| \;>\; \sum_{j\neq i} |\mathbf{N}_{ij}|,
$$
This is related to the spectral radius of the iteration matrix $\mathbf{G}$, through the induced infinity-norm:
$$
    \|\mathbf{G}\|_\infty 
    = \max_i \sum_j \bigl| (\mathbf{M}^{-1}\mathbf{N})_{ij}\bigr| 
    \;\le\;\max_i \frac{1}{|\mathbf{M}_{ii}|}\sum_{j\neq i}|\mathbf{N}_{ij}|
    \;<\;1.
$$
A basic property of induced norms is that for any eigenpair $(\lambda, v)$ of $\mathbf{G}$, for some eigenvector $v\neq0$, then:
$$
|\lambda|\;\|v\|_\infty \;=\;\|\lambda v\|_\infty
\;=\;\|\mathbf{G}v\|_\infty
\;\le\;\|\mathbf{G}\|_\infty\,\|v\|_\infty
\quad\Longrightarrow\quad
|\lambda|\le\|\mathbf{G}\|_\infty.
$$
Taking the maximum over all eigenvalues gives
$\rho(\mathbf{G})=\max_i|\lambda_i|\le\|\mathbf{G}\|_\infty.$


### Rate of convergence?
The rate of convergence of an iterative method is related to the spectral radius of the iteration matrix $\mathbf{G}$. Let's consider a _worst-case_ bound. Suppose we have some induced norm $\|\mathbf{G}\| = q$, then for any iteration $k$ we have:
$$\begin{align*}
\|\mathbf{e}^{(k)}\| & = \lVert\mathbf{G}^{k}\;\mathbf{e}^{(0)}\rVert\\
& \le\; \|\mathbf{G}^{k}\|\,\|\mathbf{e}^{(0)}\|\\
& \le\; q^{k}\,\|\mathbf{e}^{(0)}\|\\
\end{align*}
$$
To reach a desired accuracy $\epsilon$ at iteration $k$, we can bound the error as:
$$
\|\mathbf{e}^{(k)}\| \leq q^{k}\,\|\mathbf{e}^{(0)}\| \leq \epsilon
$$
which implies:
$$
\boxed{
   k\geq\frac{\ln(\epsilon/\|\mathbf{e}^{(0)}\|)}{|\ln(\lVert\mathbf{G}\rVert)|}
}\quad\blacksquare
$$

___